# 7.4 word2vec、fastTextを用いた日本語単語のベクトル表現の実装

- 本ファイルでは、日本語の単語をword2vecもしくはfastTextを使用してベクトル化する手法を解説します。

※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.4 学習目標

1.	学習済みの日本語word2vecモデルで単語をベクトル表現に変換する実装ができるようになる
2.	学習済みの日本語fastText モデルで単語をベクトル表現に変換する実装ができるようになる


# 事前準備
書籍の指示に従い、本章で使用するデータを用意します

pip install gensim


# 1. 文書を読み込んで、分かち書き、データセット作成まで（8.2と同じです）

前処理と分かち書きをし、最後にデータセットを作成する部分を実装します


In [1]:
# 単語分割にはMecab＋NEologdを使用
import MeCab

# m_t = MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')
m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret



# 前処理として正規化をする関数を定義
import re

def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    return text


# 前処理とJanomeの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Mecabの単語分割

    return ret


In [3]:
import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


# フォルダ「data」から各tsvファイルを読み込みます
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

/home/ubuntu/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


# 2. 単語のベクトル化

## 2.1 word2vec

単語をベクトル表現に変換します。

TorchTextには日本語の学習済みデータがないわけではないですが、精度が微妙なので

東北大学 乾・岡崎研究室で公開されているWord2Vecの学習済みのベクトルを使用します。



In [4]:
# そのままではtorchtextで読み込めないので、gensimライブラリを使用して、
# Word2Vecのformatで保存し直します

# 事前インストール
# pip install gensim

from gensim.models import KeyedVectors


# 一度gensimライブラリで読み込んで、word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format(
    './data/entity_vector/entity_vector.model.bin', binary=True)

# 保存（時間がかかります、10分弱）
model.wv.save_word2vec_format('./data/japanese_word2vec_vectors.vec')


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


In [5]:
# torchtextで単語ベクトルとして読み込みます
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='./data/japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_word2vec_vectors.dim)
print("単語数：", len(japanese_word2vec_vectors.itos))


100%|█████████▉| 1015237/1015474 [01:50<00:00, 8903.78it/s]

1単語を表現する次元数： 200
単語数： 1015474


100%|█████████▉| 1015237/1015474 [02:10<00:00, 8903.78it/s]

In [6]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 49個の単語が200次元のベクトルで表現されている
TEXT.vocab.vectors


torch.Size([49, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-2.8353,  2.5609, -0.5348,  ...,  0.4602,  1.4669, -2.1255],
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.0448, -0.1304,  0.0329,  ...,  0.0825, -0.1386,  0.0417]])

In [7]:
# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fb275fc18d0>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
     

In [8]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械学習 tensor(-0.1404)


姫 - 女性 + 男性　を計算すると狙った通り、王子がもっとも近い結果になりました

## 2.2 fastText

word2vecより進歩したベクトル化手法であるfastTextによる単語のベクトル表現を使用します。

日本語の学習モデルを以下の記事にて公開してくださっているので、使用させていただきます。



In [9]:
# Qiita：いますぐ使える単語埋め込みベクトルのリスト
# https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

# Download Word Vectors
# https://drive.google.com/open?id=0ByFQ96A4DgSPNFdleG1GaHcxQzA

In [13]:
# torchtextで単語ベクトルとして読み込みます
# word2vecとは異なり、すぐに読み込めます

from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='./data/vector_neologd/model.vec')

                                    
# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_fasttext_vectors.dim)
print("単語数：", len(japanese_fasttext_vectors.itos))



  0%|          | 0/351122 [00:00<?, ?it/s]Skipping token b'351122' with 1-dimensional vector [b'300']; likely a header

100%|█████████▉| 351101/351122 [00:50<00:00, 7690.32it/s]

1単語を表現する次元数： 300
単語数： 351122



100%|█████████▉| 351101/351122 [01:03<00:00, 7690.32it/s]

In [14]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 52個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


torch.Size([49, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fb25bc7a310>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
     

In [15]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3650)
王 tensor(0.3461)
王子 tensor(0.5531)
機械学習 tensor(0.0952)
